In [1]:
import json
import math
import os
import random
import re
import time
import warnings
from glob import glob

import cv2
import numpy as np
import pandas as pd
import timm
import torch
from PIL import Image
from matplotlib import image as mpimg, pyplot as plt
from sklearn.metrics import f1_score, mean_absolute_error
from sklearn.model_selection import KFold, StratifiedKFold
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from tqdm.auto import tqdm

warnings.filterwarnings('ignore')

def fix_everything(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    np.random.seed(random_seed)
    random.seed(random_seed)

fix_everything(42)

In [2]:
TARGET_IDX = 0

class LsdDataset(Dataset):
    def __init__(self, imgs, mode='train'):
        self.imgs = imgs
        self.img_contents = []

        print("Opening Images...")
        for img in tqdm(self.imgs):
          self.img_contents.append(Image.open(img).convert("RGB"))
          
        self.mode = mode
          # Try adding more augmentation:
          # WORSE: 
          # - RandomAdjustSharpness(sharpness_factor=2), 
          # - ColorJitter(brightness=0.4, contrast=0.2, saturation=0.3, hue=0.3),

        self.transforms = {
          "train":    transforms.Compose([
                      transforms.RandomHorizontalFlip(),
                      transforms.RandomVerticalFlip(),
                      transforms.ToTensor(),
                    ]),

          "valid":    transforms.Compose([
                      transforms.ToTensor(),
                    ]),
                                    
          "test":     transforms.Compose([
                      transforms.ToTensor(),
                                  ])
            } 
    
    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        # Get Image.
        img_title = self.imgs[idx]
        img = self.img_contents[idx]
        transforms = self.transforms[self.mode]
        img = transforms(img)  

        # Get label.
        matches = re.findall(r'[0-9]+\.?[0-9]*', img_title.split("/")[-1])
      
        label = [float(x) for x in matches]
        label[0] /= 6 # may change.
        label = [round(x, 4) for x in label]

        if len(label) != 3:
          assert(f"Wrong Image Title. Please check again.\n[Detected]: {img_title}")

        return img, torch.HalfTensor(label)

In [3]:
import torch.nn.functional as F

class Network(nn.Module):
    def __init__(self, model_name, p=0.25):
        super(Network, self).__init__()
        self.model = timm.create_model(model_name, pretrained=True, num_classes=3, drop_rate=0.25)
    def forward(self, img):
        final = self.model(img)
        return final

In [4]:
### MAIN SETTINGS ###
import glob


train_idxs = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
val_idxs = [20, 21]

imgs = []
val_imgs = []

for idx in train_idxs:
    imgs.extend(glob.glob(f"/workspace/LSD/new_data/LineStyleData_{idx}/*.png"))

for idx in val_idxs:
    val_imgs.extend(glob.glob(f"/workspace/LSD/new_data/LineStyleData_{idx}/*.png"))

# folds = [(train_idx, valid_idx) for train_idx, valid_idx in KFold(n_splits=5, shuffle=True, random_state=42).split(imgs)]
# Train: 1064, Valid: 267
# Original Image size: 1920 x 1080 => (256, 256)

model_name = "regnetv_064"
device = torch.device("cuda:0")
training_epochs = 25
batch_size = 16

In [5]:
# 1 - [sum of (관측치 - 모델 예상치)^2] / [sum of (관측치 - in sample 평균)^2]
import torch
import numpy as np
import pdb

FIRST_ELEM_MAX = 6
FIRST_ELEM_STEP_SIZE = 0.5

SECOND_ELEM_MAX = 1
SECOND_ELEM_STEP_SIZE = 0.1

THIRD_ELEM_MAX = 1
THIRD_ELEM_STEP_SIZE = 0.1

def process_vals(pred):
    pred[pred > 1] = 1
    pred[pred < 0]= 0

    # Since first element minimum is 1:
    pred[:, 0] = np.round(pred[:, 0] * FIRST_ELEM_MAX / FIRST_ELEM_STEP_SIZE ) * FIRST_ELEM_STEP_SIZE
    pred[:, 0][pred[:, 0] < 1] = 1

    pred[:, 1] = np.round(pred[:, 1] * SECOND_ELEM_MAX / SECOND_ELEM_STEP_SIZE) * SECOND_ELEM_STEP_SIZE
    pred[:, 2] = np.round(pred[:, 2] * THIRD_ELEM_MAX / THIRD_ELEM_STEP_SIZE) * THIRD_ELEM_STEP_SIZE
    return pred

def get_rsquared_total(pred, gt):
    # Unnormalize & Round
    pred = process_vals(pred)
    gt = process_vals(gt)

    one = get_rsquared(pred, gt, 0)
    two = get_rsquared(pred, gt, 1)
    three = get_rsquared(pred, gt, 2)
    return {"width": one, "dynamics": two, "jitter": three}

def nrmse(pred, gt):
    pred = process_vals(pred)
    gt = process_vals(gt)

    one = get_nrmse(pred, gt, 0)
    two = get_nrmse(pred, gt, 1)
    three = get_nrmse(pred, gt, 2)
    return {"width": one, "dynamics": two, "jitter": three}

def get_nrmse(pred, gt, idx):
    pred = pred[:, idx]
    gt = gt[:, idx]
    numerator = np.sqrt(np.mean((gt - pred)**2))
    denominator = np.max(gt) - np.min(gt)
    return numerator / denominator

def get_rsquared(pred, gt, idx):
    pred = pred[:, idx]
    gt =  gt[:, idx]
    r_sq = 1 - sum(np.square(gt- pred)) / sum(np.square(gt - np.mean(gt)))
    return r_sq

def weighted_mse_loss(input, target):
    return (torch.HalfTensor([1, 2, 1.5]).to(device) * (input - target) ** 2).mean()

In [6]:
### TRAINING
from tqdm.auto import tqdm
import pdb

for fold, idxs in enumerate(range(1)):
  
  model = nn.DataParallel(Network(model_name), device_ids=[0,1]) # If multi-gpu
  # model = Network(model_name) # else use this.
  
  model = model.to(device)
  scaler = torch.cuda.amp.GradScaler() 
  optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
  criterion = weighted_mse_loss

  best= 1e5

  print(f"Running fold No.{fold+1}")
  
  # train_idx, valid_idx = idxs
  # train_dataset = LsdDataset(np.array(imgs)[train_idx], mode='train')
  train_dataset = LsdDataset(np.array(imgs), mode='train')

  train_loader = DataLoader(train_dataset,
                    batch_size=batch_size, pin_memory=True, num_workers=4, shuffle=True)

  # valid_dataset = LsdDataset(np.array(imgs)[valid_idx], mode='valid')
  valid_dataset = LsdDataset(np.array(val_imgs), mode='valid')
  valid_loader = DataLoader(valid_dataset, batch_size=batch_size, pin_memory=True, num_workers=4, shuffle=True)
  
  for epoch in range(training_epochs):
      
      start=time.time()
      train_loss = 0
      
      model.train()
      optimizer.zero_grad()

      train_errors = []
      eval_errors = []

      train_vals = []
      eval_vals = []

      with tqdm(train_loader, unit="batch") as t_epoch:
        for idx, batch in enumerate(t_epoch):

          x = torch.tensor(batch[0], dtype=torch.float32, device=device) # img
          y = torch.tensor(batch[1], dtype=torch.float32, device=device) # label
          pred = model(x)
          
          loss = criterion(pred.squeeze(1), y)
          loss.backward()

          optimizer.step()
          optimizer.zero_grad()
          
          train_loss += loss.item()/len(train_loader)
          t_epoch.set_postfix(loss=loss.item())

          train_errors.append(abs(pred - y).detach().cpu())
          train_vals.append((pred.detach().cpu(), y.detach().cpu()))

      model.eval()
      valid_loss = 0
      
      with torch.no_grad():
           with tqdm(valid_loader, unit="batch") as v_epoch:
             for batch in v_epoch:
              x = torch.tensor(batch[0], dtype=torch.float32, device=device)
              y = torch.tensor(batch[1], dtype=torch.float32, device=device)
              
              pred = model(x)
              loss = criterion(pred.squeeze(1), y)
              valid_loss += loss.item()/len(valid_loader)
              v_epoch.set_postfix(loss=loss.item())
              eval_errors.append(abs(pred - y).detach().cpu())
              eval_vals.append((pred.detach().cpu(), y.detach().cpu()))
            
      TIME = time.time() - start
      train_score = abs(torch.cat(train_errors, dim=0)).mean(axis=0)
      valid_score = abs(torch.cat(eval_errors, dim=0)).mean(axis=0)

      if  sum(valid_score) <= best:
          best = sum(valid_score)
          # But it's convsmall, in reality.x``
          torch.save(model.state_dict(), f'/workspace/LSD/result/regnet0207.pth')

      print(f'epoch : {epoch+1}/{training_epochs}    time : {TIME:.0f}s/{TIME*(training_epochs-epoch-1):.0f}s')
      print(f'Train Loss: {sum(train_score):.5f}')
      print(f'Valid Loss: {sum(valid_score):.5f} best : {best:.5f}')
      print(f"NRMSE Valid: {nrmse(torch.cat([x[0] for x in eval_vals]).numpy(), torch.cat([x[1] for x in eval_vals]).numpy())}")
      print(f"R-Squared Valid: {get_rsquared_total(torch.cat([x[0] for x in eval_vals]).numpy(), torch.cat([x[1] for x in eval_vals]).numpy())}")

Running fold No.1
Opening Images...


  0%|          | 0/21296 [00:00<?, ?it/s]

Opening Images...


  0%|          | 0/2662 [00:00<?, ?it/s]

  0%|          | 0/1331 [00:03<?, ?batch/s]

...

  0%|          | 0/1331 [00:01<?, ?batch/s]

  0%|          | 0/167 [00:01<?, ?batch/s]

epoch : 16/25    time : 598s/5381s
Train Loss: 0.07873
Valid Loss: 0.25219 best : 0.12336
NRMSE Valid: {'width': 0.08307122, 'dynamics': 0.1384686, 'jitter': 0.087733805}
R-Squared Valid: {'width': 0.9309917355371901, 'dynamics': 0.8082644654520466, 'jitter': 0.9230278014208003}


  0%|          | 0/1331 [00:01<?, ?batch/s]

KeyboardInterrupt: 